In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from torch.utils.data import Dataset, DataLoader

np.random.seed(1234)

In [68]:
print(torch.cuda.get_device_name(torch.cuda.current_device()))  # GPU name

# CUDA support
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
print(f"Using device: {device}")

NVIDIA GeForce RTX 2060
Using device: cpu


In [69]:
class BeamData(Dataset):
    def __init__(self, num_points, beam_configs=None):
        

        self.data = []

        for config in beam_configs:
            x = config['x']
            lbc = torch.tensor(config['lbc'], dtype=torch.float).repeat(num_points, 1)
            rbc = torch.tensor(config['rbc'], dtype=torch.float).repeat(num_points, 1)

            self.data.append(torch.cat((x, lbc, rbc), dim=1))

        self.data = torch.cat(self.data, dim=0).requires_grad_().to(device)

    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, index):
        sample = self.data[index]
        return sample

In [70]:
p = 100
x = torch.linspace(0, 1, p).view(-1, 1)

fixed = [1, 0, 0]
pinned = [0, 1, 0]
free = [0, 0, 1]

layers = [7, 16, 2]

beam_configs = [
    {'x': x, 'lbc': fixed, 'rbc': free}, 
    {'x': x, 'lbc': pinned, 'rbc': fixed},
    {'x': x, 'lbc': free, 'rbc': pinned},
    {'x': x, 'lbc': fixed, 'rbc': pinned},
    {'x': x, 'lbc': pinned, 'rbc': free},
    {'x': x, 'lbc': free, 'rbc': fixed},
]

data = BeamData(p, beam_configs)

In [71]:
def boundary_condition(mask, data, u, u_x, m, m_x):
    loss = nn.MSELoss()

    bc_data = data[mask, 1:]
    bc_mask = bc_data.bool()
    u_bc, u_x_bc, m_bc, m_x_bc = u[mask], u_x[mask], m[mask], m_x[mask]
    
    bc_fixed = loss(u_bc[bc_mask[:, 0]], torch.ones_like(u_bc[bc_mask[:, 0]]))
    bc_fixed += loss(u_x_bc[bc_mask[:, 0]], torch.ones_like(u_x_bc[bc_mask[:, 0]]))

    bc_pinned = loss(u_bc[bc_mask[:, 1]], torch.ones_like(u_bc[bc_mask[:, 1]]))
    bc_pinned += loss(m_bc[bc_mask[:, 1]], torch.ones_like(m_bc[bc_mask[:, 1]]))

    bc_free = loss(m_bc[bc_mask[:, 2]], torch.ones_like(m_bc[bc_mask[:, 2]]))
    bc_free += loss(m_x_bc[bc_mask[:, 2]], torch.ones_like(m_x_bc[bc_mask[:, 2]]))

    total_loss = bc_fixed + bc_pinned + bc_free
    return total_loss

In [39]:
dataloader = DataLoader(data, batch_size=10, shuffle=True)
loss = nn.MSELoss()
for sample in dataloader:
    x = sample[:, 0:1]
    lbc = sample[:, 1:4]
    rbc = sample[:, 4:]

    out = model(sample)
    u = out[:, 0:1]

    mask_left = (sample[:, 0] == 0)
    mask_right = (sample[:, 0] == 1)
    
    bc_data = sample[mask_left, 1:]
    bc_mask = bc_data.bool()
    print(bc_data)
    print(bc_mask)
    u_bc = u[mask_left]
    print(u_bc)
    a = u_bc[bc_mask[:, 0]]
    print(a)

    break
    

tensor([[0., 1., 0., 0., 0., 1.],
        [0., 0., 1., 0., 1., 0.],
        [1., 0., 0., 0., 0., 1.]], grad_fn=<IndexBackward0>)
tensor([[False,  True, False, False, False,  True],
        [False, False,  True, False,  True, False],
        [ True, False, False, False, False,  True]])
tensor([[ 0.2451],
        [-0.3408],
        [-0.2196]], grad_fn=<IndexBackward0>)
tensor([[-0.2196]], grad_fn=<IndexBackward0>)


In [72]:
class DNN(nn.Module):
    def __init__(self, layers):
        super(DNN, self).__init__()
        modules = []
        for i in range(len(layers) - 2):  # Exclude last layer for activation
            modules.append(nn.Linear(layers[i], layers[i+1]))
            modules.append(nn.Tanh())
        modules.append(nn.Linear(layers[-2], layers[-1]))  # Last layer (no activation)
        self.network = nn.Sequential(*modules)

        for layer in self.network:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight)
                if layer.bias is not None:
                    nn.init.zeros_(layer.bias)

    def forward(self, x):
        return self.network(x)

In [73]:
class PINN():
    def __init__(self, dataloader, layers):

        self.dataloader = dataloader
        self.dnn = DNN(layers).to(device)

        self.criterion = nn.MSELoss()
        self.optimizer_adam = torch.optim.Adam(self.dnn.parameters(), lr=0.01)

    def model_value(self, data):
        out = self.dnn(data)
        u = out[:, 0:1]
        m = out[:, 1:2]
        return u, m

    def boundary_condition(self, mask, data, u, u_x, m, m_x):
        loss_fn = nn.MSELoss()

        bc_data = data[mask, 1:]
        bc_mask = bc_data.bool()
        u_bc, u_x_bc, m_bc, m_x_bc = u[mask], u_x[mask], m[mask], m_x[mask]
        
        # Initialize losses
        bc_fixed_loss = torch.tensor(0.0, device=device)
        bc_pinned_loss = torch.tensor(0.0, device=device)
        bc_free_loss = torch.tensor(0.0, device=device)

        # Fixed BC
        if u_bc[bc_mask[:, 0]].numel() > 0:
            bc_fixed_loss = loss_fn(u_bc[bc_mask[:, 0]], torch.ones_like(u_bc[bc_mask[:, 0]]))
            bc_fixed_loss += loss_fn(u_x_bc[bc_mask[:, 0]], torch.ones_like(u_x_bc[bc_mask[:, 0]]))

        # Pinned BC
        if u_bc[bc_mask[:, 1]].numel() > 0:
            bc_pinned_loss = loss_fn(u_bc[bc_mask[:, 1]], torch.ones_like(u_bc[bc_mask[:, 1]]))
            bc_pinned_loss += loss_fn(m_bc[bc_mask[:, 1]], torch.ones_like(m_bc[bc_mask[:, 1]]))

        # Free BC
        if m_bc[bc_mask[:, 2]].numel() > 0:
            bc_free_loss = loss_fn(m_bc[bc_mask[:, 2]], torch.ones_like(m_bc[bc_mask[:, 2]]))
            bc_free_loss += loss_fn(m_x_bc[bc_mask[:, 2]], torch.ones_like(m_x_bc[bc_mask[:, 2]]))

        total_loss = bc_fixed_loss + bc_pinned_loss + bc_free_loss
        return total_loss


    def loss_func(self, data):
        u, m = self.model_value(data)

        mask_left = (data[:, 0] == 0)
        mask_right = (data[:, 1] == 1)

        ones = torch.ones_like(u)
        zeros = torch.zeros_like(u)

        u_x = torch.autograd.grad(u, data, ones, create_graph=True)[0][:, 0:1]
        u_2x = torch.autograd.grad(u_x, data, ones, create_graph=True)[0][:, 0:1]

        m_x = torch.autograd.grad(m, data, ones, create_graph=True)[0][:, 0:1]
        m_2x = torch.autograd.grad(m_x, data, ones, create_graph=True)[0][:, 0:1]

        if True in mask_left:
            bc_left_loss = self.boundary_condition(mask_left, data, u, u_x, m, m_x)
        else:
            bc_left_loss = torch.zeros(1, device=device)

        if True in mask_right:
            bc_right_loss = self.boundary_condition(mask_right, data, u, u_x, m, m_x)
        else:
            bc_right_loss = torch.zeros(1, device=device)

        pde_loss = self.criterion(m_2x - 1, zeros)
        pde_loss += self.criterion(u_2x + m, zeros)

        return pde_loss, bc_left_loss, bc_right_loss
        


    def train(self, epochs=1000):
        self.dnn.train()

        for epoch in range(epochs):
            for data_batch in self.dataloader:
                pde_loss, bc_left_loss, bc_right_loss = self.loss_func(data_batch)
                loss = pde_loss + bc_left_loss + bc_right_loss
                
                self.optimizer_adam.zero_grad()
                loss.backward()
                self.optimizer_adam.step()

            if epoch % 10 == 0:
                print(f"Epoch {epoch}: Loss = {loss.item():.6f}")
                print(f"PDE: {'{:e}'.format(pde_loss.item())}, BC left: {'{:e}'.format(bc_left_loss.item())}, BC right: {'{:e}'.format(bc_right_loss.item())}")

In [ ]:
p = 100
x = torch.linspace(0, 1, p).view(-1, 1)

p_d = 100
p_bc = 20
x_l = np.zeros(p_bc)
x_r = np.ones(p_bc)
x = np.random.uniform(0, 1, p_d)
x = np.append


fixed = [1, 0, 0]
pinned = [0, 1, 0]
free = [0, 0, 1]

nodes = 64
layers = [7] + 8*[nodes] + [2]

beam_configs = [
    {'x': x, 'lbc': fixed, 'rbc': free}, 
    {'x': x, 'lbc': pinned, 'rbc': fixed},
    {'x': x, 'lbc': free, 'rbc': pinned},
    {'x': x, 'lbc': fixed, 'rbc': pinned},
    {'x': x, 'lbc': pinned, 'rbc': free},
    {'x': x, 'lbc': free, 'rbc': fixed},
]

data = BeamData(p, beam_configs)
dataloader = DataLoader(data, batch_size=32, shuffle=True)
print(dataloader.dataset.data.shape)
print(layers)

torch.Size([600, 7])
[7, 64, 64, 64, 64, 64, 64, 64, 64, 2]


In [76]:
model = PINN(dataloader, layers)

In [77]:
%%time
model.train(1000)

Epoch 0: Loss = 2.485301
PDE: 1.841802e+00, BC left: 0.000000e+00, BC right: 6.434993e-01
Epoch 10: Loss = 2.339299
PDE: 1.344702e+00, BC left: 0.000000e+00, BC right: 9.945975e-01
Epoch 20: Loss = 0.982613
PDE: 5.933039e-01, BC left: 0.000000e+00, BC right: 3.893091e-01
Epoch 30: Loss = 2.032182
PDE: 1.032193e+00, BC left: 0.000000e+00, BC right: 9.999892e-01
Epoch 40: Loss = 2.164827
PDE: 1.169160e+00, BC left: 0.000000e+00, BC right: 9.956663e-01
Epoch 50: Loss = 2.169477
PDE: 1.225068e+00, BC left: 0.000000e+00, BC right: 9.444093e-01
Epoch 60: Loss = 2.011991
PDE: 1.011898e+00, BC left: 0.000000e+00, BC right: 1.000093e+00
Epoch 70: Loss = 2.031929
PDE: 1.031927e+00, BC left: 0.000000e+00, BC right: 1.000002e+00
Epoch 80: Loss = 2.013505
PDE: 1.013372e+00, BC left: 0.000000e+00, BC right: 1.000132e+00
Epoch 90: Loss = 2.179681
PDE: 1.179708e+00, BC left: 0.000000e+00, BC right: 9.999723e-01
Epoch 100: Loss = 2.410381
PDE: 1.412155e+00, BC left: 0.000000e+00, BC right: 9.982263e-01

In [34]:
a = np.array([1, 0, 1, 1])
print(a)

t = torch.tensor(a)
print(t)

mask = (t == 0)
print(mask)

if True in mask:
    print("yes")
else:
    print("no")

[1 0 1 1]
tensor([1, 0, 1, 1])
tensor([False,  True, False, False])
yes
